In [ ]:
#@title mount gdrive and laod settings (run once)
import os.path
from os import path

from google.colab import drive
drive.mount('/content/drive')

gdrive_root = "/content/drive/MyDrive"
gdrive_path = "/stable-diffusion" #@param {type:"string"}

!mkdir -p {gdrive_root}{gdrive_path}

#@markdown Change these for new models
model = "sd-v1-4.ckpt" #@param {type:"string"}
hf_path = "https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt" #@param {type:"string"}

#@markdown You need to get a token from here https://huggingface.co/settings/tokens
#@markdown It will load from settings.yml if exists
hf_token = "" #@param {type:"string"}

!pip install ruamel.yaml

import sys 
from ruamel.yaml import YAML

yaml=YAML(typ='safe')

with open(gdrive_root + gdrive_path + '/settings.yml', 'r') as yaml_t:
    data = yaml.load(yaml_t)
    model = data["model"] if 'model' in data else model
    hf_path = data["hf_path"] if 'hf_path' in data else hf_path
    hf_token = data['hf_token'] if 'hf_token' in data else hf_token

sdui="/content/stable-diffusion-webui"

print(model)
print(hf_path)
print(hf_token)
print(sdui)

In [ ]:
#@title setup (run once)

!mkdir -p {gdrive_root}{gdrive_path}/models

if path.exists(gdrive_root + gdrive_path + "/models/" + model) == False:
    user_header = f"\"Authorization: Bearer {hf_token}\""
    !wget --header={user_header} {hf_path} -O {gdrive_root}{gdrive_path}/models/{model}

if path.exists(gdrive_root + gdrive_path + "/models/" + "GFPGANv1.4.pth") == False:
    !wget https://github.com/TencentARC/GFPGAN/releases/download/v1.3.4/GFPGANv1.4.pth \
    -O {gdrive_root}{gdrive_path}/models/GFPGANv1.4.pth

!nvidia-smi

!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui {sdui}

%cd {sdui}
!pip install -r requirements.txt

!mkdir repositories
!git clone https://github.com/CompVis/stable-diffusion.git repositories/stable-diffusion
!git clone https://github.com/CompVis/taming-transformers.git repositories/taming-transformers
!git clone https://github.com/sczhou/CodeFormer.git repositories/CodeFormer
!git clone https://github.com/salesforce/BLIP.git repositories/BLIP
!pip install -r repositories/CodeFormer/requirements.txt
!git clone https://github.com/crowsonkb/k-diffusion repositories/k-diffusion
!pip install clip
!pip install gdown

def replace_line(file_name, line_num, text):
    lines = open(file_name, 'r').readlines()
    lines[line_num] = text
    out = open(file_name, 'w')
    out.writelines(lines)
    out.close()

def replace_line_in_file(file_name, line_to_search, text_to_replace):
    with open(file_name, 'r') as file:
        # read a list of lines into data
        data = file.readlines()

    for line in data:
        # if the line contains the string we're looking for,
        # write the line to the output file
        if line_to_search in line:
            replace_line(file_name, data.index(line), text_to_replace)

%cd {sdui}/modules
replace_line_in_file('sd_models.py', 'pl_sd = torch.load(checkpoint_file, map_location="cpu")', '    pl_sd = torch.load(checkpoint_file, map_location="cuda:0")\n')

!ln -sf {gdrive_root}{gdrive_path}/models/{model} {sdui}/models/Stable-diffusion/model.ckpt
!ln -sf {gdrive_root}{gdrive_path}/models/GFPGANv1.4.pth {sdui}/GFPGANv1.4.pth 
!ln -sf {gdrive_root}{gdrive_path}/outputs {sdui}/outputs

In [ ]:
#@title run 
!python {sdui}/webui.py --share